<a href="https://colab.research.google.com/github/s1300211/Guraduation-Thesis/blob/master/Instructions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SROOEを用いたDEMの超解像手順


-----

## **1. DEM float 32 GeoTIFF → tensor_rgb → SROOE に直接入力**
* **LQ_dataset.py** の `__getitem__` を編集
* **PNG 読み込み削除**
* **rasterio で float32 GeoTIFF 読み込み**
* **1ch → 3ch 複製でモデル入力に合わせる**

#### *修正前*
```
# get LQ image
LQ_path = self.paths_LQ[index]
img_LQ = util.read_img(self.LQ_env, LQ_path)
H, W, C = img_LQ.shape

if self.opt['color']:
    img_LQ = util.channel_convert(C, self.opt['color'], [img_LQ])[0]

if img_LQ.shape[2] == 3:
    img_LQ = img_LQ[:, :, [2, 1, 0]]
img_LQ = torch.from_numpy(np.ascontiguousarray(np.transpose(img_LQ, (2, 0, 1)))).float()

return {'LQ': img_LQ, 'LQ_path': LQ_path}
```

#### *修正後*
```
def __getitem__(self, index):
    import rasterio
    import numpy as np
    import torch

    # DEM GeoTIFF のパス
    LQ_path = self.paths_LQ[index]

    # ---- DEM 読み込み (float32) ----
    with rasterio.open(LQ_path) as src:
        dem = src.read(1).astype(np.float32)    # shape: (H, W)

    # ---- min-max 正規化（preprocess.py と同じ処理）----
    dem_min, dem_max = dem.min(), dem.max()
    dem_norm = (dem - dem_min) / (dem_max - dem_min + 1e-8)

    # ---- 1ch → 3ch に複製 (SROOE は in_nc=3 が標準) ----
    # test.ymlでは in_nc=4 だが、RGB入力を前提にしているため3chでOK
    dem_3ch = np.stack([dem_norm, dem_norm, dem_norm], axis=0)  # (3, H, W)

    # ---- tensor 化 ----
    img_LQ = torch.from_numpy(dem_3ch).float()

    return {'LQ': img_LQ, 'LQ_path': LQ_path}
```

<br>

## **test.yml の設定**
SROOEの最初の状態では、RRDBNet の入力チャンネル数が４になっている。
```
network_G:
  in_nc: 4
```

しかし、普通の SROOE/ESRGAN 系は 3ch (RGB) を前提としているので変更すべき
```
network_G:
  in_nc: 3
```





-----

## **2. 拡張子 .tif を入力として認識させる**
* **util.py** に修正を加える。
* 認識する拡張子として .tif を追加することで、DEMを入力ファイルとして認識するようになる。
#### *修正前（先頭付近　13~14行）*
```
IMG_EXTENSIONS = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP']
```
#### *修正後（`.tif` `.tiff` を追加）*
```
IMG_EXTENSIONS = ['.jpg', '.JPG', '.jpeg', '.JPEG',
                  '.png', '.PNG', '.ppm', '.PPM',
                  '.bmp', '.BMP', '.tif', '.tiff', '.TIF', '.TIFF']
```

-----
## **3. PNG を経由せず、tensor を float32 のまま GeoTIFF で保存する関数を追加**
* 今のままでは、超解像後は PNG として保存される。
* DEM の高さ値（高制度 float32 ）が失われる
* **test.py** に以下の処理を追加：
  * **visuals['SR'] を flaot 32 numpy array で取得**
  * **rasterio を使って TIFF で書き出し**
  * **元の DEM の geotransform & crs をそのままコピー**

<br>

### **3-1. test.py へ関数を追加**
**test.py** の先頭に以下の処理を追加
```
import rasterio
from rasterio.transform import Affine
```
<br>

###**3-2. DEM を保存する関数を test.py に追加**
test.py の上の方に置いてOK
```
def save_sr_as_dem(sr_tensor, ref_dem_path, save_path):
    """
    sr_tensor: shape (1, H, W) or (3, H, W) の torch.Tensor（float32）
    ref_dem_path: 元TIF (GeoTIFF)
    save_path: 保存先 TIF path
    """

    import numpy as np
    import rasterio

    sr_np = sr_tensor.squeeze().cpu().numpy()  # → (H, W)

    # 元のDEMの座標系・transform をコピー
    with rasterio.open(ref_dem_path) as src:
        profile = src.profile.copy()

    # 出力は float32 の DEM
    profile.update(
        dtype=rasterio.float32,
        count=1
    )

    with rasterio.open(save_path, 'w', **profile) as dst:
        dst.write(sr_np.astype(np.float32), 1)
```
<br>

### **3-3. test.py の保存処理を上書き**  

#### *修正前*
現在 test.py には次の部分があります（現在 PNG 保存部分）
```
sr_img = util.tensor2img(visuals['SR'])  # uint8
cm_img = util.tensor2img(visuals['CM'])  # uint8

# save images
suffix = opt['suffix']
if suffix:
    save_img_path = osp.join(dataset_dir, img_name + suffix + '.png')
else:
    save_img_path = osp.join(dataset_dir, img_name + '.png')
util.save_img(sr_img, save_img_path)

save_cm_path = os.path.join(dataset_dir, '{:s}_cmap.png'.format(img_name))
util.save_img(cm_img, save_cm_path)

logger.info('{:20s}'.format(img_name))
```

これを完全削除して次のようにします。
#### *修正後*
```
# --- SR tensor を float32 のまま取得 ---
sr_tensor = visuals['SR']   # shape: (1,1,H,W) or (1,3,H,W)

# 入力DEM（LQ path）を参照として使う
ref_dem_path = data['LQ_path'][0]

# 必要なら 3ch → 1ch へ
if sr_tensor.shape[1] == 3:
    sr_tensor = sr_tensor.mean(dim=1, keepdim=True)

# 保存先（GeoTIFF）
save_tif_path = osp.join(dataset_dir, img_name + '.tif')

# --- DEM として保存 ---
save_sr_as_dem(sr_tensor, ref_dem_path, save_tif_path)

logger.info(f"Saved DEM: {save_tif_path}")
```